# Pipeline Testing Document

In [1]:
import astroPHD
from astroPHD.util.inspect import getfullerargspec
from decorator import decorator
from astroPHD.pipeline.pipeline_function import PipelineFunction

In [2]:
def exfunc(x, y, a=.1, b=.2, *args, ll='l', mm='m', **kwargs):
    """exfunc doc"""
    print(x, y, '  ', a, b, '  ', args, '  ', ll, mm, '  ', kwargs)
    return x, y+a
# /def

def exfunc2(x, z, *, ll='l', nn='n'):
    """exfunc2 doc"""
    print(x, z, '  ', a, c, '  ', '  ', ll, nn, '  ')
    return x + z
# /def

In [3]:
import importlib
importlib.reload(astroPHD.pipeline.pipeline_function)
from astroPHD.pipeline.pipeline_function import PipelineFunction

PipelineFunction?

Init signature: PipelineFunction(self, func, outargnames, inargnames=None, name=None, **kw)
Docstring:     
Node of a Pipeline

INFO
----
The signature is that of the underlying function
The defaults for the pipeline can be seen in self._defaults  # TODO better method

Parameters
----------
func : function

outargnames: tuple, None
    the names of the outputs of *func*
    ** only used by next function in a Pipeline **
    mandatory b/c python cannot introspect function returns
    when used in a Pipeline, this is used by *inargnames* of the next
    function, unless otherwise specified / overwritten

inargnames : dict  # TODO
    ** only used by previous function in a Pipeline **
    for a dictionary of inputs *indict*, *inargnames* maps the keys of
    *indict* to the (kw)args of *func*
    *func* is introspected for the full set of possible keys, anything
    not explicitly remapped by *inargnames* is assumed to be the same.
        ex: for func(x, y, z) & inargnames = dict(x='a')


In [4]:
FMfunc = PipelineFunction(exfunc, ('x', 'ypa'), {'x': 'a'}, a=.11, x=2.1, ll='ll', args=(1, 2))
print(FMfunc._outargnames)
print(FMfunc._inargnames)

FMfunc(1, 2, a=4)

('x', 'ypa')
{'x': 'a'}
1 2    4 0.2    ()    l m    {}


(1, 6)

In [5]:
@PipelineFunction.decorator('xpy')
def FMfunc(x, y, a=.1, b=.2, *args, ll='l', mm='m', **kwargs):
    """FMfu
    nc doc"""
    print(x, y, '  ', a, b, '  ', args, '  ', ll, mm, '  ', kwargs)
    return x + y
# /def

FMfunc?

res = FMfunc.run(x=0, y=1, args=(2, 3, 5), kwtest='test')
print(res)

0 1    0.1 0.2    (2, 3, 5)    l m    {'kwtest': 'test'}
{'xpy': 1}


Signature:      FMfunc(x, y, a=0.1, b=0.2, *args, ll='l', mm='m', **kwargs)
Docstring:     
FMfu
nc doc
Init docstring:
Node of a Pipeline

Defaults
--------
The defaults for the pipeline can be seen in self._defaults  # TODO better method

PipelineFunction Parameters
---------------------------
File:           ~/src/astroPHD/pipeline/<ipython-input-5-fe631676365b>
Type:           PipelineFunction


# Pipeline

In [6]:
@PipelineFunction.decorator(('x', 'ypa'),
                            inargnames={'x': 'x'},
                            name='FUNCTION 1',
                            y=.2)
def func1(x, y, a=.1, b=.2, *args, ll='l', mm='m', **kwargs):
    """exfunc doc"""
    return x, y+a
# /def

@PipelineFunction.decorator(('z2'), inargnames={'ypa': 'z'},
                            name='FUNCTION 2')
def func2(x, z, *args, ll='l', nn='n'):
    """exfunc2 doc"""
    return z + 2
# /def

# func1.run(x=1)

In [7]:
func1.name


'FUNCTION 1'

In [8]:
class Pipeline(object):

    def __init__(self, *steps):
        """
        steps : list
            action class needs a .run method
        """

        self.steps = steps
        # TODO process functions which are not PipelineFunctions as (name, func)
        # TODO steps should be a multi-linked list

        return
    # /def

    def run(self, startkw={}, **stepsargs):
        """
        """
        
        # TODO stepsargs

        kw = startkw  # input for first step

        if set(startkw.keys()).difference(self.steps[0]._inargnames):
            raise ValueError

        for step in self.steps:

#             stepkw = {k: kw[k] for k in step._inargnames if k in kw}
            stepkw = kw  # TODO some check that a) passing right args b) not passing extra args?
            stepkw.update(stepsargs.get(step.name, {}))

            print(step.name, stepkw, step._defaults, step._outargnames)

            kw = step.run(**stepkw)

            print('kw:', kw)

        return kw
    # /def

pipe = Pipeline(func1, func2)

pipe.run({'x': 1})


FUNCTION 1 {'x': 1} {'y': 0.2} ('x', 'ypa')
FUNCTION 1 {'x': 1} {'y': 0.2}
kw: {'x': 0.2, 'ypa': 1.1}
FUNCTION 2 {'x': 0.2, 'ypa': 1.1} {} ('z2',)
FUNCTION 2 {'x': 0.2, 'z': 1.1} {}
kw: {'z2': 3.1}


{'z2': 3.1}